In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import *
import astropy.units as u

import sys,os
sys.path.append(os.path.join(os.getcwd(),".."))
#jupyter的运行路径是文件所在路径，这里要添加上一级的路径到系统变量才能import utils
from utils import ball
from tqdm import tqdm
from astropy.io import fits

In [2]:
file_path="J_AJ_146_86_table1.dat.gz.fits"
with fits.open(file_path) as hdul:
    hdul.info()
    
    column=hdul[1].data.columns
    table=hdul[1].data

Filename: J_AJ_146_86_table1.dat.gz.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      70   ()      
  1  table1.dat    1 TableHDU       326   8315R x 48C   [I7, F6.2, F5.2, F4.2, A8, I2, I2, F4.1, A1, I2, I2, F4.1, F8.4, F8.4, F8.4, F8.4, I2, F4.2, F5.2, F5.2, I5, I5, I5, I5, I5, A10, I4, I4, I3, F6.2, F5.2, F5.3, I3, F8.4, F8.4, F8.4, F8.4, I2, F5.2, F5.2, I5, I5, I5, I5, I5, I4, I7, A15]   


In [3]:
def coordinate(row):
    # z=d*np.sin(np.deg2rad(la))
    # x=d*np.cos(np.deg2rad(la))*np.cos(np.deg2rad(lo))
    # y=d*np.cos(np.deg2rad(la))*np.sin(np.deg2rad(lo))
    # return d,lo,la,np.array([x,y,z]).reshape(1,3)
    c=SkyCoord(
        sgl = row[1]*u.degree,
        sgb = row[2]*u.degree,
        distance = row[0]*ball.scale_factor(5)*u.mpc,
        frame = "supergalactic"
    )
    return [c.cartesian.x.value,c.cartesian.y.value,c.cartesian.z.value]

coordinate_matrix=np.stack([table["Dist"],table["SGLON"],table["SGLAT"]], axis=1)
coordinate_matrix=np.apply_along_axis(coordinate, 1, coordinate_matrix)
coordinate_matrix=np.array(coordinate_matrix)
print(coordinate_matrix[:5])

[[ 12926.86806207 -12457.76252495   5989.96438116]
 [ 21814.73594583 -14112.69360349   9341.86369409]
 [ 28684.45797551 -30493.86318207  13414.12642431]
 [ 27285.54098964 -21185.58968687  11983.48393398]
 [ 13120.27252995 -26615.46814786   7285.41409186]]


In [4]:
def name(PGC,Name,Group):
    s="PGC"+str(PGC)+" "
    if Name:
        s+=str(Name)+" "
    if Group:
        s+=str(Group)
    return s.strip()

name_list=[]
for row in tqdm(table):
    name_list.append( name(row["PGC"],row["Name"],row["Group"]))

print(name_list[:5])

100%|██████████| 8315/8315 [00:00<00:00, 24894.15it/s]

['PGC4 AGC331060', 'PGC55 UGC12898', 'PGC70 UGC12900', 'PGC76 UGC12901', 'PGC124 UGC12913']


In [ ]:
for i in range(5):
    print(name_list[i],coordinate_matrix[i])

In [ ]:
ball.generateBallObj(coordinate_matrix,os.path.join("./", os.path.basename(file_path).split(".")[0]))

In [5]:
# looking for Andromeda
for i in range(len(name_list)):
    if "PGC2557" in name_list[i]:
        print(name_list[i])
        print(coordinate_matrix[i])


PGC2557 NGC0224 222
[ 257.29709324 -113.51855479   62.61472581]
PGC25570 NGC2763 522
[-2836.37657702  4790.03854894 -9012.11513007]
